In [2]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time

In [ ]:
def crawl(url, depth,visited):
    if depth < 2 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        time.sleep(0.25)
        response = requests.get(url,headers=headers)
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
        except:
            soup = BeautifulSoup(response.text, 'lxml')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [urljoin(url, link) for link in links if link]

        for link in links:
            if link not in visited:
                visited.add(link)
                #if link.startswith(url):
                crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://news.kapook.com'
website_dict3 = crawl(base_url, 0, set())
print(website_dict3)

In [ ]:
def check_domain(base_url,links):
    result= set()
    for link in links :
        if link.startswith(base_url):
            result.add(link)
    return result

websitedict4=check_domain(base_url,website_dict3)

In [ ]:
def check_not_domain(base_url,links):
    result= set()
    for link in links :
        if not link.startswith(base_url):
            result.add(link)
    return result

not_domain = check_not_domain(base_url,website_dict3)
print(not_domain)

In [1]:
target_links= {'https://www.thairath.co.th':0,'https://news.kapook.com':0}
def check_ref(base,links):
    for i in links:
        for j in base:
            if i.startswith(j):
                target_links[base_url]+=1
    return target_links

ref=check_ref(target_links,not_domain)
print(ref)
           

NameError: name 'not_domain' is not defined

In [ ]:
target_links= {'https://www.thairath.co.th':0,'https://news.kapook.com':0}
list_targey_links =list(target_links)
print(list_targey_links)

In [ ]:
class spyder:
    def __init__( self ,target_links,base_url ):
        self.base_url = base_url
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.check_domain_result = self.check_domain(self.base_url,self.get_crawler())
        return self.check_domain_result
    
    def get_check_not_domain(self):
        self.check_not_domain_result = self.check_not_domain(self.base_url,self.get_crawler())   
        return self.check_not_domain_result
    
    def get_check_ref(self):
        self.check_ref_result = self.check_ref(self.base_url,self.get_check_not_domain(),target_links)
        return self.check_ref_result
    
    def get_all(self):
        crawl = self.crawl(self.base_url,0,set())
        check_domain =  self.check_domain(self.base_url,self.get_crawler()) 
        check_not_domain = self.check_domain(self.base_url,self.get_crawler())
        check_ref = self.check_ref(self.base_url,self.get_check_not_domain(),target_links)
        return crawl,check_domain,check_not_domain,check_ref
    
    def crawl(self,url, depth,visited):
        if depth < 1 :
            visited.add(url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            time.sleep(0.3)
            response = requests.get(url,headers=headers)
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                soup = BeautifulSoup(response.text, 'lxml')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    visited.add(link)
                    #if link.startswith(url):
                    crawl(link,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(self,base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result
    
    def check_not_domain(self,base_url,links):
        result= set()
        for link in links :
            if not link.startswith(base_url):
                result.add(link)
        return result
    
    def check_ref(self,base,links,target_links):
        for i in links:
            for j in target_links:
                if i.startswith(j):
                    target_links[j]+=1
        return target_links

In [73]:
target_links = {'https://www.thairath.co.th':0,'https://news.kapook.com':0}
for i in target_links:
    web_spyder=spyder(target_links,i)
    print(web_spyder.get_check_ref())
    print('------------------------------------------------------------------------------------------------------------------')

{'https://www.thairath.co.th': 0, 'https://news.kapook.com': 0}
------------------------------------------------------------------------------------------------------------------
{'https://www.thairath.co.th': 2, 'https://news.kapook.com': 0}
------------------------------------------------------------------------------------------------------------------


In [ ]:
base_url = 'https://www.thairath.co.th'
target_links=['https://www.thairath.co.th','https://www.facebook.com']
website = crawl(base_url, 0, visited=set())
domain = check_domain(base_url,website)
not_domain = check_not_domain(base_url,website)
ref = check_ref(target_links,not_domain)
print(website)
print('-------------------------------------------')
print(domain)
print('-------------------------------------------')
print(not_domain)
print('-------------------------------------------')
print(ref)